In [ ]:
pip install scikit-learn

In [1]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.probability import FreqDist
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
import time
from google.cloud import storage
import os
import pandas as pd

In [2]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "development-416403-b306695feb67.json"

In [24]:
# open file and 
start_time = time.time()
lemmatizer = WordNetLemmatizer()
files = open('llm.txt', 'r')

text = files.read()

# chunk to word
words = [word.lower() for word in text.split() if word.isalpha()]

# remove the stopwords
stop_words = set(stopwords.words('english'))

without_stop_word_file = [word for word in words if word not in stop_words]


#  store the most commom word in list
most_common_word_list = []

for most_common_word, _ in FreqDist(without_stop_word_file).most_common(10):
    most_common_word_list.append(most_common_word)


print("most_common_words:", most_common_word_list)
    # vectorize the list of the most common word
tfid = TfidfVectorizer()
text_vector =tfid.fit_transform(most_common_word_list)
# print(tfid.get_feature_names_out())
print(text_vector.todense())

# time taken for execution
end_time = time.time()
elapsed_time = end_time - start_time
print("time taken:",elapsed_time)
# time taken: 0.006171464920043945

most_common_words: ['language', 'llms', 'clinical', 'patient', 'documentation', 'data', 'text', 'clinicians', 'including', 'content']
[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]]
time taken: 0.2108461856842041


In [ ]:
pip list


In [11]:
def clean_data (text):


    # lemmatizer = WordNetLemmatizer(text)
    # chunk to word
    words = [word.lower() for word in text.split() if word.isalpha()]

    # remove the stopwords
    stop_words = set(stopwords.words('english'))

    without_stop_word_file = [word for word in words if word not in stop_words]


    #  store the most commom word in list
    most_common_word_list = []

    for most_common_word, _ in FreqDist(without_stop_word_file).most_common(10):
        most_common_word_list.append(most_common_word)
    
    # print("most_common_words:", most_common_word_list)
    return most_common_word_list

        # vectorize the list of the most common word
    # tfid = TfidfVectorizer()
    # text_vector =tfid.fit_transform(most_common_word_list)
    # print(tfid.get_feature_names_out())
    # print(text_vector.todense())
    # time taken for execution


read the file from bucket
split it to word
remove stopwords
lemmatize
get most common word 
store it in form of a dic the file name as key and common words as value

In [14]:
# dic for storing most common words in each file
start_time_n = time.time()
dic_most_common_word = dict()
# development_awarri/oratory/english/src-docs
# fetch file from the bucket
def fetch_files (bucket_path):
    star_fn = time.time()
    storage_client = storage.Client()
    bucket_name = "development_awarri"
    # prefix = "read/english/src-docs"
    prefix = bucket_path

    source_bucket = storage_client.bucket(bucket_name)
    blobs = storage_client.list_blobs(bucket_name, prefix=prefix, delimiter=None)
    for blob in blobs:
        file = blob.name
        dic_key = file.split("/")[-1]
        blob_file = source_bucket.blob(file)
        with blob_file.open("r") as f:

            # read file by line
            text = f.read()
            
            # using function to clean and get the most common words
            common_words = clean_data(text)
        #    storeing the result into dictionary with file name as key
            dic_most_common_word[dic_key] = common_words
            
fetch_files(bucket_path ='oratory/english/src-docs')   
end_time_n = time.time()
print(dic_most_common_word)
print("time taken:", end_time_n-start_time_n)

{'  OBASANJO Voting Rights March Commemoration Speech.txt': ['got', 'going', 'joshua', 'people', 'us', 'sure', 'something', 'still', 'generation', 'folks'], ' 15th BRICS Summit: Nigeria Remains Committed to Collaborating with China, Others – VP Shettima.txt': ['china', 'african', 'president', 'vice', 'areas', 'south', 'africa', 'partnership', 'initiatives', 'include'], ' 15th BRICS Summit: President Tinubu’s Economic Reforms, Diplomatic Alliances to Attract Investment and Partnership to Nigeria— VP Shettima.txt': ['president', 'global', 'vice', 'south', 'international', 'world', 'brics', 'africa', 'partnership', 'governance'], ' 53rd Independence speech by President Jonathan.txt': ['us', 'every', 'nation', 'must', 'nigeria', 'political', 'may', 'years', 'independence', 'people'], ' A Call for Unity and Freedom in Nigeria.txt': ['nigerian', 'nigeria', 'one', 'divisive', 'forces', 'national', 'would', 'freedom', 'world', 'today'], ' A KEYNOTE ADDRESS DELIVERED BY CHIEF (DR.) J. O. SANUSI

In [15]:
# Original dictionary with filenames as keys and lists of words as values
original_dict = {
    'file1.txt': ['apple', 'banana', 'orange'],
    'file2.txt': ['grape', 'pineapple', 'watermelon'],
    'file3.txt': ['cherry', 'strawberry', 'blueberry']
}

# Convert to a new dictionary where keys are filenames and values are flattened lists of words
flattened_dict = {key: words for key, words_list in original_dict.items() for words in [words_list]}

print(flattened_dict)


{'file1.txt': ['apple', 'banana', 'orange'], 'file2.txt': ['grape', 'pineapple', 'watermelon'], 'file3.txt': ['cherry', 'strawberry', 'blueberry']}


In [ ]:
# export to cwd as csv
dic_most_common_word.to_csv('name to save with', index = False)

Analysis on the common words


Read domain

Step

1. load the data
2. get information about the dataset
3. remove remove empty cells
4. check for wrong data format
5. check for wrong data
6. Check for duplicates and remove 

In [4]:
# load the dataset

read_df = pd.read_csv('read-most-common-words.csv')
read_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,"200 BDC operators arrested, EFCC alleges frau...",arrested,suspects,efcc,forex,enugu,one,financial,oyewale,state,charge
1,A National Eulogy in Remembering the Fallen H...,us,national,names,world,nation,prayers,many,god,nigerians,life
2,African Night's Entertainment.txt,novel,abu,child,kyauta,old,story,vengeance,mallam,different,make
3,Awọn oṣiṣẹ ileewe ijọba ipinlẹ Ọyọ binu si Go...,n,ti,awọn,wọn,ṣe,ni,ileewe,oṣiṣẹ,ijọba,lawọn
4,Did Tinubu appoint retired AIG Odumosu substa...,odumosu,efcc,government,appointed,retired,tinubu,social,media,official,statement
...,...,...,...,...,...,...,...,...,...,...,...
2155,zulum.txt,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2156,‘He helped Nigeria with P&ID case’ – Lai mourn...,lamorde,nigeria,former,minister,said,institutional,advertisement,court,billion,information
2157,‘Multilateralism – Today’s Imperative’.txt,international,world,approach,global,challenges,need,countries,greater,multilateral,problems
2158,"‘Olu Jacobs is alive’ – Betty Irabor, family r...",rumours,jacobs,founder,genevieve,betty,debunked,news,making,rounds,nollywood


In [5]:
#  2. get information about the dataset 

read_df.info()
read_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2160 entries, 0 to 2159
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2160 non-null   object
 1   0           2144 non-null   object
 2   1           2144 non-null   object
 3   2           2144 non-null   object
 4   3           2144 non-null   object
 5   4           2144 non-null   object
 6   5           2144 non-null   object
 7   6           2144 non-null   object
 8   7           2144 non-null   object
 9   8           2144 non-null   object
 10  9           2144 non-null   object
dtypes: object(11)
memory usage: 185.8+ KB


,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,"200 BDC operators arrested, EFCC alleges frau...",arrested,suspects,efcc,forex,enugu,one,financial,oyewale,state,charge
1,A National Eulogy in Remembering the Fallen H...,us,national,names,world,nation,prayers,many,god,nigerians,life
2,African Night's Entertainment.txt,novel,abu,child,kyauta,old,story,vengeance,mallam,different,make
3,Awọn oṣiṣẹ ileewe ijọba ipinlẹ Ọyọ binu si Go...,n,ti,awọn,wọn,ṣe,ni,ileewe,oṣiṣẹ,ijọba,lawọn
4,Did Tinubu appoint retired AIG Odumosu substa...,odumosu,efcc,government,appointed,retired,tinubu,social,media,official,statement


In [6]:
#  3. remove remove empty cells

new_read_df = read_df.dropna()
new_read_df.info()  


<class 'pandas.core.frame.DataFrame'>
Index: 2144 entries, 0 to 2159
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  2144 non-null   object
 1   0           2144 non-null   object
 2   1           2144 non-null   object
 3   2           2144 non-null   object
 4   3           2144 non-null   object
 5   4           2144 non-null   object
 6   5           2144 non-null   object
 7   6           2144 non-null   object
 8   7           2144 non-null   object
 9   8           2144 non-null   object
 10  9           2144 non-null   object
dtypes: object(11)
memory usage: 201.0+ KB


In [ ]:
# new_read_df.columns = ['Filepath'] + list(new_read_df.columns[1:])  # Assign new column names

# Method 2: Using dictionary for renaming (alternative)
# new_read_df = new_read_df.rename(columns={'original_column_name': 'Filepath'})  # Replace with actual original name


In [15]:
# 4. check for wrong data format
# 5. check for wrong data

# Change the first column to filepath
new_read_df = new_read_df.rename(columns={'Unnamed: 0': 'Filepath'})

# remove wrong data
new_read_df.drop(3, inplace=True)

new_read_df.head()



,Filepath,0,1,2,3,4,5,6,7,8,9
0,"200 BDC operators arrested, EFCC alleges frau...",arrested,suspects,efcc,forex,enugu,one,financial,oyewale,state,charge
1,A National Eulogy in Remembering the Fallen H...,us,national,names,world,nation,prayers,many,god,nigerians,life
2,African Night's Entertainment.txt,novel,abu,child,kyauta,old,story,vengeance,mallam,different,make
4,Did Tinubu appoint retired AIG Odumosu substa...,odumosu,efcc,government,appointed,retired,tinubu,social,media,official,statement
5,Echoes of Tomorrow.txt,charlotte,book,within,pages,existed,forgotten,obscured,amidst,old,one


In [17]:
# Check for duplicates and remove
new_read_df.duplicated()

new_read_df.drop_duplicates()
print(new_read_df.duplicated())

0       False
1       False
2       False
4       False
5       False
        ...  
2154    False
2156    False
2157    False
2158    False
2159    False
Length: 2143, dtype: bool


In [18]:
# import the cleaned file
new_read_df.to_csv('treated_read.csv')

Oratory domain


Step

1. load the data
2. get information about the dataset
3. remove remove empty cells
4. check for wrong data format
5. check for wrong data
6. Check for duplicates and remove 

In [3]:
oratory_df = pd.read_csv('oratory-most-common-words.csv')
oratory_df

,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,OBASANJO Voting Rights March Commemoration S...,got,going,joshua,people,us,sure,something,still,generation,folks
1,15th BRICS Summit: Nigeria Remains Committed ...,china,african,president,vice,areas,south,africa,partnership,initiatives,include
2,15th BRICS Summit: President Tinubu’s Economi...,president,global,vice,south,international,world,brics,africa,partnership,governance
3,53rd Independence speech by President Jonatha...,us,every,nation,must,nigeria,political,may,years,independence,people
4,A Call for Unity and Freedom in Nigeria.txt,nigerian,nigeria,one,divisive,forces,national,would,freedom,world,today
...,...,...,...,...,...,...,...,...,...,...,...
3007,“This agreement must make sure that the women ...,women,also,climate,know,make,gender,agreement,change,un,world
3008,“Through sport we can teach some of life’s big...,women,sports,also,goals,gender,sustainable,development,us,able,men
3009,“We have opportunities as much as we have chal...,work,also,us,women,countries,make,would,commit,young,un
3010,“What Matters More Than Your Talents”.txt,gifts,years,per,would,wanted,follow,life,grandparents,grandmother,number


In [4]:
#  2. get information about the dataset 

oratory_df.info()
oratory_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3012 entries, 0 to 3011
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3012 non-null   object
 1   0           3010 non-null   object
 2   1           3010 non-null   object
 3   2           3010 non-null   object
 4   3           3010 non-null   object
 5   4           3010 non-null   object
 6   5           3008 non-null   object
 7   6           3007 non-null   object
 8   7           3006 non-null   object
 9   8           3006 non-null   object
 10  9           3005 non-null   object
dtypes: object(11)
memory usage: 259.0+ KB


,Unnamed: 0,0,1,2,3,4,5,6,7,8,9
0,OBASANJO Voting Rights March Commemoration S...,got,going,joshua,people,us,sure,something,still,generation,folks
1,15th BRICS Summit: Nigeria Remains Committed ...,china,african,president,vice,areas,south,africa,partnership,initiatives,include
2,15th BRICS Summit: President Tinubu’s Economi...,president,global,vice,south,international,world,brics,africa,partnership,governance
3,53rd Independence speech by President Jonatha...,us,every,nation,must,nigeria,political,may,years,independence,people
4,A Call for Unity and Freedom in Nigeria.txt,nigerian,nigeria,one,divisive,forces,national,would,freedom,world,today


In [5]:
#  3. remove remove empty cells

new_oratory_df = oratory_df.dropna()
new_oratory_df.info()  


<class 'pandas.core.frame.DataFrame'>
Index: 3005 entries, 0 to 3011
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  3005 non-null   object
 1   0           3005 non-null   object
 2   1           3005 non-null   object
 3   2           3005 non-null   object
 4   3           3005 non-null   object
 5   4           3005 non-null   object
 6   5           3005 non-null   object
 7   6           3005 non-null   object
 8   7           3005 non-null   object
 9   8           3005 non-null   object
 10  9           3005 non-null   object
dtypes: object(11)
memory usage: 281.7+ KB


In [6]:
# 4. check for wrong data format
# 5. check for wrong data

# Change the first column to filepath
new_oratory_df = new_oratory_df.rename(columns={'Unnamed: 0': 'Filepath'})



new_oratory_df.head()

,Filepath,0,1,2,3,4,5,6,7,8,9
0,OBASANJO Voting Rights March Commemoration S...,got,going,joshua,people,us,sure,something,still,generation,folks
1,15th BRICS Summit: Nigeria Remains Committed ...,china,african,president,vice,areas,south,africa,partnership,initiatives,include
2,15th BRICS Summit: President Tinubu’s Economi...,president,global,vice,south,international,world,brics,africa,partnership,governance
3,53rd Independence speech by President Jonatha...,us,every,nation,must,nigeria,political,may,years,independence,people
4,A Call for Unity and Freedom in Nigeria.txt,nigerian,nigeria,one,divisive,forces,national,would,freedom,world,today


In [7]:
# Check for duplicates and remove
new_oratory_df.duplicated()

new_oratory_df.drop_duplicates()
print(new_oratory_df.duplicated())

0       False
1       False
2       False
3       False
4       False
        ...  
3007    False
3008    False
3009    False
3010    False
3011    False
Length: 3005, dtype: bool


In [8]:
# import the cleaned file
new_oratory_df.to_csv('treated_oratory.csv')